In [2]:
import os
import sys
sys.path.append('..')

In [3]:
import pandas as pd
import numpy as np
time_horizons = [64,128, 256]

all_errors = None
all_mu = None
for time_horizon in time_horizons:
  data_dir = f'../processed_data_{time_horizon}'
  df = pd.read_csv(os.path.join(data_dir, 'all_errors.csv'),  index_col=0, parse_dates=True)
  # compute the mean of the errors
  avg_err = df.mean(axis=1).mean()
  df = df * np.sqrt(256 / time_horizon)
  print(f'Average error for time horizon {time_horizon}: {avg_err}')

  df = df.add_suffix(f'_{time_horizon}')
  mu = np.load(os.path.join(data_dir, 'mu.npy'))
  mu = mu * (256 / time_horizon)

  if all_errors is None:
    all_errors = df
  else:
    all_errors = pd.concat([all_errors, df], axis=1, join='outer')

  if all_mu is None:
    all_mu = mu
  else:
    all_mu = np.concatenate([all_mu, mu])


Average error for time horizon 64: 0.006851094008759641
Average error for time horizon 128: 0.03363520184794312
Average error for time horizon 256: 0.02844700424454357


In [4]:
print(all_errors.shape)
print(all_mu.shape)

(1461, 2222)
(2222,)


In [5]:
# find out the indice of assets in mu that is less than mean(mu)
mean_mu = all_mu.mean()
idx = np.where(all_mu < 0)[0]
print(f'Number of assets with mu < mean(mu): {len(idx)}')
print(idx)

Number of assets with mu < mean(mu): 750
[   0    1    3    4    6    9   11   12   15   18   19   24   25   27
   29   31   32   33   35   36   48   50   54   55   56   59   65   68
   71   75   76   77   79   80   82   85   89   91   92   93   94   96
  100  103  104  105  109  110  111  114  115  119  120  122  123  124
  133  135  136  137  138  148  152  153  161  164  171  178  182  183
  185  187  190  192  193  206  207  210  212  214  218  220  228  231
  234  236  237  241  246  249  252  258  260  264  265  266  267  273
  278  280  286  289  292  295  297  301  305  307  312  313  315  317
  327  332  334  335  342  343  346  351  353  354  355  356  358  361
  383  385  398  404  415  418  419  426  442  458  463  469  476  487
  493  494  502  504  510  513  514  523  524  531  542  543  550  551
  559  566  568  569  571  581  585  586  595  596  597  598  600  602
  606  608  609  611  612  615  616  617  622  625  626  629  631  637
  639  644  645  646  647  650  668 

In [6]:

# remove the assets with mu < mean(mu) in both of mu and all_errors
# all_mu = np.delete(all_mu, idx)
# all_errors = all_errors.drop(all_errors.columns[idx], axis=1)

In [7]:
print(len(all_mu))
print(all_errors.shape)

2222
(1461, 2222)


In [8]:
from sklearn.covariance import LedoitWolf

def get_shrinkage_covariance(df):
    lw = LedoitWolf(store_precision=False, assume_centered=True)
    lw.fit(df)
    # Convert the ndarray back to a DataFrame and use the column and index from the original DataFrame
    shrink_cov = pd.DataFrame(lw.covariance_, index=df.columns, columns=df.columns)
    return shrink_cov

all_errors = all_errors.fillna(method='ffill').fillna(method='bfill')
S = get_shrinkage_covariance(all_errors)

/tmp/ipykernel_75225/3200124498.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_errors = all_errors.fillna(method='ffill').fillna(method='bfill')


In [9]:
S.shape

(2222, 2222)

In [10]:
S.shape

(2222, 2222)

In [11]:
import torch
def project_capped_simplex(v, z=1.0, u=0.1):
    # 1) sort v in descending order
    u_sorted, _ = torch.sort(v, descending=True)
    # 2) find the threshold theta for the simplex proj
    cssv = torch.cumsum(u_sorted, dim=0) - z
    ind = torch.arange(1, len(v)+1, device=v.device, dtype=v.dtype)
    cond = u_sorted - cssv/ind > 0
    rho = ind[cond][-1]
    theta = cssv[cond][-1] / rho
    # 3) project onto simplex
    w = torch.clamp(v - theta, min=0.0)
    # 4) enforce box constraint
    w = torch.clamp(w, max=u)
    # 5) re-project to simplex if sum ≠ z
    if abs(w.sum().item() - z) > 1e-6:
        # another round of simplex projection:
        w_sorted, _ = torch.sort(w, descending=True)
        cssv2 = torch.cumsum(w_sorted, dim=0) - z
        cond2 = w_sorted - cssv2/ind > 0
        rho2 = ind[cond2][-1]
        theta2 = cssv2[cond2][-1] / rho2
        w = torch.clamp(w - theta2, min=0.0)
    return w

In [ ]:
import torch

torch.set_num_threads(2)
# 1. Prepare data: mu (expected returns) and sigma (std deviations)
#    Suppose you already have two 1D NumPy arrays of length N=4000:
#      mu_np    = np.array([...], dtype=np.float32)
#      sigma_np = np.array([...], dtype=np.float32)
#    Here we just illustrate with random data.
N = all_mu.shape[0]
mu    = torch.tensor(all_mu, dtype=torch.float32)  # e.g. expected returns
sigma = torch.tensor(S.values, dtype=torch.float32)  # e.g. covariance matrix

# 2. Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mu    = mu.to(device)
sigma = sigma.to(device)

# 3. Initialize weights (unconstrained parameter)
initial_guess = N * [1. / N]
raw_w = torch.tensor(initial_guess, dtype=torch.float32, device=device)
raw_w = torch.nn.Parameter(raw_w)

# 4. Set up optimizer
optimizer = torch.optim.Adam([raw_w], lr=1e-4)

# 5. (Optional) Control number of CPU threads if on CPU


# Compile the model
compiled_model = torch.compile(model)
# 6. Optimization loop
for step in range(3000):
    optimizer.zero_grad()
    # a) apply box constraints [0, 0.1]
    w = project_capped_simplex(raw_w)
    #w_clamped = torch.clamp(raw_w, min=0.0, max=0.1)
    # b) enforce sum-to-one

    #w = w_clamped / (w_clamped.sum() + 1e-12)

    # c) compute portfolio return and risk
    port_ret  = torch.dot(w, mu)
    port_var = w @ sigma @ w
    port_risk = torch.sqrt(port_var + 1e-12)

    # d) Sharpe ratio (no risk-free rate assumed)
    #sharpe = port_ret / port_risk
    one_sigma = port_ret - port_risk
    # e) maximize Sharpe → minimize negative Sharpe
    #loss = -sharpe
    loss = -one_sigma
    loss.backward()
    optimizer.step()

    # (Optional) print progress
    if step % 200 == 0:
        print(f"Step {step:4d}  one_sigma = {one_sigma.item():.4f}")

# 7. Extract optimized weights on CPU
w_opt = (torch.clamp(raw_w, 0.0, 0.1) / torch.clamp(raw_w, 0.0, 0.1).sum()).detach()
w_opt = w_opt.cpu().numpy()
# print non-zero weights
non_zero_weights = np.where(w_opt > 0)[0]
print(f'Number of non-zero weights: {len(non_zero_weights)}')
print(f'Number of zero weights: {N - len(non_zero_weights)}')
print(f'Non-zero weights: {w_opt[non_zero_weights]}')
print(f'final return: {np.dot(w_opt, all_mu)}')
print(f'final risk: {np.sqrt(w_opt @ S.values @ w_opt)}')

# Now w_opt is your final weight vector of length 4000, summing to 1 with each ≤ 0.1.


TypeError: torch.FloatTensor is not a Module subclass

In [14]:
def portfolio_volatility_log_return(weights, covariance):
    return np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

def portfolio_log_return(weights, returns):
    return np.sum(returns*weights)

print("log return",portfolio_log_return(w_opt, all_mu))
print("volatility", portfolio_volatility_log_return(w_opt, S.values))


log return 0.6259529384490016
volatility 0.2673337869401924


In [17]:
from scipy.optimize import minimize


def get_bounds(N, lower_bound, upper_bound):
  # for ETF, the allowed weight is between 0 and 20%
  # for stocks, the allowed weight is between 0 and 10%
  num_assets = N
  if num_assets == 0:
    return None

  bounds = tuple((lower_bound, upper_bound) for asset in range(num_assets))
  return bounds

def min_func_one_sigma(weights, returns, covariance, risk_free_rate):
  portfolio_ret = portfolio_log_return(weights, returns)
  portfolio_vol = portfolio_volatility_log_return(weights, covariance)
  return -(portfolio_ret - risk_free_rate - portfolio_vol)


def min_func_sharpe(weights, returns, covariance, risk_free_rate):
    portfolio_ret = portfolio_log_return(weights, returns)
    portfolio_vol = portfolio_volatility_log_return(weights, covariance)
    sharpe_ratio = (portfolio_ret - risk_free_rate) / portfolio_vol
    return -sharpe_ratio # Negate Sharpe ratio because we minimize the function

def optimize_portfolio(returns, covariance, risk_free_rate, bounds):
    num_assets = len(returns)
    args = (returns, covariance, risk_free_rate)

    # Define constraints
    def constraint_sum(weights):
        return np.sum(np.abs(weights)) - 1

    constraints = [{'type': 'eq', 'fun': constraint_sum}]


    # Perform optimization
    def objective(weights):
        return min_func_one_sigma(weights, returns, covariance, risk_free_rate)

    iteration = [0]  # mutable container to store iteration count
    def callback(weights):
        iteration[0] += 1

        print(f"Iteration: {iteration[0]}, value: {objective(weights)}")

    # Initial guess (equal weights)
    initial_guess = num_assets * [1. / num_assets]

    # Perform optimization
    result = minimize(objective, initial_guess,
                      method='SLSQP', bounds=bounds, constraints=constraints, callback=callback, options={'maxiter': 10})

    return result

In [18]:
bounds = get_bounds(all_mu.shape[0], 0, 0.1)
raw_weights = optimize_portfolio(all_mu, S, 0, bounds)
print('final return: ', portfolio_log_return(raw_weights.x, all_mu))
print('final risk: ', portfolio_volatility_log_return(raw_weights.x, S.values))
print('final weights: ', raw_weights.x)

Iteration: 1, value: -0.3155814297010861
Iteration: 2, value: -0.3449851843670154
Iteration: 3, value: -0.3560713526037627
Iteration: 4, value: -0.3577201986370235
Iteration: 5, value: -0.358071357987287
Iteration: 6, value: -0.3582133781268702
Iteration: 7, value: -0.3583134038206965
Iteration: 8, value: -0.3583271881336771
Iteration: 9, value: -0.3583336935040428
Iteration: 10, value: -0.3583336935040428
final return:  0.6224007586024712
final risk:  0.2640670650984284
final weights:  [0.00000000e+00 0.00000000e+00 5.04896354e-15 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


In [20]:
non_zero_weights = np.where(raw_weights.x > 0.005)[0]
print(f'Non-zero weights: {raw_weights.x[non_zero_weights]}')

Non-zero weights: [0.02842462 0.1        0.09987548 0.0817379  0.06508591 0.1
 0.08406259 0.0472043  0.1        0.02979073 0.06381847 0.1
 0.1       ]
